In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np
import seaborn as sns
from sklearn import datasets, linear_model

/Users/arnavverma/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import c3aidatalake

In [3]:
survey = c3aidatalake.fetch(
    "surveydata",
    {
        "spec": {
            "filter": ""
        }
    },
    get_all = True
)

ConnectionError: HTTPSConnectionPool(host='api.c3.ai', port=443): Max retries exceeded with url: /covid/api/1/surveydata/fetch (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fe00cdfbe90>: Failed to establish a new connection: [Errno 65] No route to host'))

In [ ]:
pd.set_option('display.max_columns', None)
#Change birthyear to age for better understanding
survey = survey.rename(columns={"birthYear2020": "Age"})
survey['Age'] = 2020 - survey['Age']
#drop duplicates and take out any survey entries not in United States (US focus)
survey.drop_duplicates()
survey = survey.rename(columns={"location.id": "Location"})
survey = survey[survey['Location'].str.endswith('_UnitedStates')]
survey['Location'] = survey['Location'].str.slice(stop=-13)
#no null values in Location, coronaAllHospitalize, coronaKillsMost, ethnicitySpreadsCovid,
#allSpreadCovid, nonNativesSpreadCovid, asymptomaticSpread, onlySickSpread,infectFromAnimal
#intent of wearing mask is 50 if not reported (no way of knowing if a person would wear mask or not) 3987 null values
survey['coronavirusIntent_Mask'] = survey['coronavirusIntent_Mask'].fillna(50.0)
#250 did not respond to knowing someone in community that has coronavirus
#if reported greater than 5000 then dropped to 5000
#because it is unreasonable to know more than 5000 people that have been affected
#outliers have heavy impact on data
#166 out of 33000 is insignificant
#None values in surveys data replaced with mean afterwards
survey['coronavirusLocalCommunity'] = np.where((survey.coronavirusLocalCommunity > 5000),
                                               5000,
                                               survey.coronavirusLocalCommunity)
survey['coronavirusLocalCommunity'] = survey['coronavirusLocalCommunity'].fillna(np.mean(survey['coronavirusLocalCommunity']))
survey = survey.rename(columns={"birthYear2020": "Age", "annualHouseholdIncome":"HouseIncome"})
#https://www.census.gov/library/publications/2020/demo/p60-270.html#:~:text=Median%20household%20income%20was%20%2468%2C703,and%20Table%20A%2D1).
#people that did not respond with survey 
survey['HouseIncome'] = survey['HouseIncome'].fillna(68703)

In [ ]:
survey

In [ ]:

survey

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
#regr.fit(survey['coronavirusConcern'], survey['HouseIncome'])
survey.loc[survey['coronavirusIntent_Mask'] > 50, 'coronavirusIntent_Mask'] = 1.789564567
survey['coronavirusIntent_Mask']

In [ ]:
plt.scatter(survey['coronavirusIntent_Mask'][:1000], survey['coronavirusConcern'][:1000])